In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
pd.__version__

'2.2.2'

In [2]:
# Read January data
df_january = pd.read_parquet('data/yellow_tripdata_2023-01.parquet')

In [3]:
df_january.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [4]:
num_columns = df_january.shape[1]
print("Number of columns:", num_columns)

Number of columns: 19


In [5]:
df_january['duration'] = df_january.tpep_dropoff_datetime - df_january.tpep_pickup_datetime
df_january['duration'] = df_january.duration.apply(lambda td: td.total_seconds() / 60)

In [6]:
# Calculate standard deviation
std_duration = df_january['duration'].std()
print("Standard Deviation of duration:", std_duration)

Standard Deviation of duration: 42.59435124195458


In [7]:
df_january.duration.describe()

count    3.066766e+06
mean     1.566900e+01
std      4.259435e+01
min     -2.920000e+01
25%      7.116667e+00
50%      1.151667e+01
75%      1.830000e+01
max      1.002918e+04
Name: duration, dtype: float64

In [8]:
# Drop outliers
df_filtered = df_january[(df_january['duration'] >= 1) & (df_january['duration'] <= 60)]
# Calculate the fraction of records left
fraction_left = len(df_filtered) / len(df_january)
print("Fraction of records left:", fraction_left)
len(df_filtered), len(df_january)

Fraction of records left: 0.9812202822125979


(3009173, 3066766)

In [11]:
df_filtered.loc[:, 'PULocationID'] = df_filtered['PULocationID'].astype(str)
df_filtered.loc[:, 'DOLocationID'] = df_filtered['DOLocationID'].astype(str)

In [12]:
dicts = df_filtered[['PULocationID', 'DOLocationID']].to_dict(orient='records')

In [13]:
dicts

[{'PULocationID': '161', 'DOLocationID': '141'},
 {'PULocationID': '43', 'DOLocationID': '237'},
 {'PULocationID': '48', 'DOLocationID': '238'},
 {'PULocationID': '138', 'DOLocationID': '7'},
 {'PULocationID': '107', 'DOLocationID': '79'},
 {'PULocationID': '161', 'DOLocationID': '137'},
 {'PULocationID': '239', 'DOLocationID': '143'},
 {'PULocationID': '142', 'DOLocationID': '200'},
 {'PULocationID': '164', 'DOLocationID': '236'},
 {'PULocationID': '141', 'DOLocationID': '107'},
 {'PULocationID': '234', 'DOLocationID': '68'},
 {'PULocationID': '79', 'DOLocationID': '264'},
 {'PULocationID': '164', 'DOLocationID': '143'},
 {'PULocationID': '138', 'DOLocationID': '33'},
 {'PULocationID': '33', 'DOLocationID': '61'},
 {'PULocationID': '79', 'DOLocationID': '186'},
 {'PULocationID': '90', 'DOLocationID': '48'},
 {'PULocationID': '113', 'DOLocationID': '255'},
 {'PULocationID': '237', 'DOLocationID': '239'},
 {'PULocationID': '143', 'DOLocationID': '229'},
 {'PULocationID': '137', 'DOLocat

In [14]:
from sklearn.feature_extraction import DictVectorizer

dv = DictVectorizer()
X_train = dv.fit_transform(dicts)

In [15]:
num_columns = X_train.shape[1]
print("Dimensionality of the matrix:", num_columns)

Dimensionality of the matrix: 515


In [16]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, df_filtered['duration'])

LinearRegression()

In [17]:
y_pred_train = model.predict(X_train)

In [18]:
from sklearn.metrics import mean_squared_error
import numpy as np

rmse_train = np.sqrt(mean_squared_error(df_filtered['duration'], y_pred_train))
print("RMSE on training data:", rmse_train)

RMSE on training data: 7.649261931416412


In [23]:
df_february = pd.read_parquet('data/yellow_tripdata_2023-02.parquet')
df_february.loc[:, 'duration'] = (df_february['tpep_dropoff_datetime'] - df_february['tpep_pickup_datetime']).dt.total_seconds() / 60
df_february_filtered = df_february[(df_february['duration'] >= 1) & (df_february['duration'] <= 60)]
df_february_filtered.loc[:, 'PULocationID'] = df_february_filtered['PULocationID'].astype(str)
df_february_filtered.loc[:, 'DOLocationID'] = df_february_filtered['DOLocationID'].astype(str)
dicts_february = df_february_filtered[['PULocationID', 'DOLocationID']].to_dict(orient='records')
X_val = dv.transform(dicts_february)

/tmp/ipykernel_168932/930093776.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['142' '132' '161' ... '158' '79' '161']' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  df_february_filtered.loc[:, 'PULocationID'] = df_february_filtered['PULocationID'].astype(str)
/tmp/ipykernel_168932/930093776.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['163' '26' '145' ... '143' '162' '140']' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  df_february_filtered.loc[:, 'DOLocationID'] = df_february_filtered['DOLocationID'].astype(str)


In [24]:
y_pred_val = model.predict(X_val)
rmse_val = np.sqrt(mean_squared_error(df_february_filtered['duration'], y_pred_val))
print("RMSE on validation data:", rmse_val)

RMSE on validation data: 7.8118162035401735
